In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import cv2
import json

In [2]:
file = open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/wordEmbeddings.json")
dict = json.load(file)

In [3]:
datasetDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset")
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset")

In [4]:
dfs = []
for file in datasetDir:
    df = pd.read_csv(file)
    dfs.append(df)
DF = pd.concat(dfs)
DF

,Unnamed: 0.1,Unnamed: 0,text,x1,y1,x2,y2,Class,invoice,invoice_X,...,mre_Y,offi,offi_X,offi_Y,CandidatePosition,NeighbourText,NeighbourX,NeighbourY,Similarity,Output
0,0,0,11-02-2019,0.132269,0.230242,0.203820,0.244355,invoice_date,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.264534,-0.014922,-0.121061,-0.011986,0.884820,0
1,1,1,19-02-2019,0.136830,0.254032,0.208381,0.268145,invoice_date,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.276369,-0.014920,-0.121088,-0.011880,0.896655,0
2,2,2,19-02-2019,0.890821,0.277823,0.962372,0.291935,invoice_date,-0.038794,0.104475,...,0.000000,0.000000,0.000000,0.000000,-0.619586,-0.001480,-0.109864,-0.011817,0.000700,1
3,3,3,19-02-2019,0.136830,0.254032,0.208381,0.268145,invoice_date,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.276369,-0.014920,-0.121088,-0.011880,0.896655,0
4,4,4,19-02-2019,0.890821,0.277823,0.962372,0.291935,invoice_date,-0.038794,0.104475,...,0.000000,0.000000,0.000000,0.000000,-0.619586,-0.001480,-0.109864,-0.011817,0.000700,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,0,15'S-15,0.235649,0.373962,0.287024,0.399738,invoice_date,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.260819,-0.006628,-0.104638,0.007134,0.578775,1
0,0,0,1-Feb-23,0.744252,0.060217,0.817265,0.069064,invoice_date,-0.038794,0.188584,...,0.000000,0.000000,0.000000,0.000000,-0.603649,0.000472,-0.066817,-0.006951,0.000718,1
0,0,0,1st,0.485691,0.086684,0.503426,0.093242,invoice_date,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-0.238035,-0.014695,-0.120251,-0.011753,0.254424,0
1,1,1,31/01/2023,0.296252,0.160251,0.366385,0.169376,invoice_date,-0.038794,0.235389,...,0.066581,0.020392,0.168480,0.059453,0.013881,0.005529,-0.090107,-0.012536,0.002508,1


In [5]:
dropCols = []
for col in dict.keys():
    if DF[col].max()==0 and DF[col].min()==0:
        dropCols.append(col)
        dropCols.append(col+"_X")
        dropCols.append(col+"_Y")
len(dropCols)

KeyError: 'renault'

In [6]:
validColumns = []
for key in dict.keys():
    if key not in dropCols:
        validColumns.append(key)
len(validColumns)


860

In [7]:
pwd

"/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset"

In [8]:
CI = []
NTI = []
NXI = []
NYI = []

In [9]:
for file in datasetDir:
    df = pd.read_csv(file)
    df.drop(columns=dropCols, inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)
    for index in df.index:
        ci = []
        nti = []
        nxi = []
        nyi = []

        ci.append(df["x1"][index])
        ci.append(df["y1"][index])
        ci.append(df["x2"][index])
        ci.append(df["y2"][index])

        for key in validColumns:
            nti.append(df[key][index])
            nxi.append(df[key+"_X"][index])
            nyi.append(df[key+"_Y"][index])

        CI.append(ci)
        NTI.append(nti)
        NXI.append(nxi)
        NYI.append(nyi)

In [10]:
CIpca = PCA(n_components=1)
NTIpca = PCA(n_components=1)
NXIpca = PCA(n_components=1)
NYIpca = PCA(n_components=1)
CIpca.fit_transform(CI)
NTIpca.fit_transform(NTI)
NXIpca.fit_transform(NXI)
NYIpca.fit_transform(NYI)

array([[-1.19862821e-02],
       [-1.18795604e-02],
       [-1.18168329e-02],
       [-1.18795604e-02],
       [-1.18168329e-02],
       [-8.66295726e-03],
       [ 1.56470219e-02],
       [-8.66295726e-03],
       [ 1.56470219e-02],
       [-1.14278334e-02],
       [-1.42659232e-02],
       [-1.19130006e-02],
       [-1.21911742e-02],
       [-1.24149850e-02],
       [-9.80132585e-03],
       [-1.18021369e-02],
       [-1.35692047e-02],
       [-1.55411968e-02],
       [ 9.93154850e-04],
       [-3.81568136e-03],
       [ 8.41760836e-04],
       [-1.16434263e-02],
       [-7.31562693e-03],
       [-1.18045432e-02],
       [-8.77396516e-03],
       [ 2.42548442e-03],
       [-8.77396516e-03],
       [ 2.42548442e-03],
       [ 5.46551716e-03],
       [-1.18019023e-02],
       [ 8.75026118e-03],
       [-8.25680424e-03],
       [ 5.46551716e-03],
       [-1.18019023e-02],
       [ 8.75026118e-03],
       [-8.25680424e-03],
       [-9.21244227e-03],
       [-1.02664566e-02],
       [-7.2

In [11]:
for file in datasetDir:
    df = pd.read_csv(file)
    df.drop(columns=dropCols, inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)
    df["CandidatePosition"] = 0
    df["NeighbourText"] = 0
    df["NeighbourX"] = 0
    df["NeighbourY"] = 0
    for index in df.index:
        ci = []
        nti = []
        nxi = []
        nyi = []

        ci.append(df["x1"][index])
        ci.append(df["y1"][index])
        ci.append(df["x2"][index])
        ci.append(df["y2"][index])

        for key in validColumns:
            nti.append(df[key][index])
            nxi.append(df[key+"_X"][index])
            nyi.append(df[key+"_Y"][index])

        df["CandidatePosition"][index] = CIpca.transform([ci])[0]
        df["NeighbourText"][index] = NTIpca.transform([nti])[0]
        df["NeighbourX"][index] = NXIpca.transform([nxi])[0]
        df["NeighbourY"][index] = NYIpca.transform([nyi])[0]
    
    validX = []
    validY = []
    for col in validColumns:
        validX.append(col+"_X")
        validY.append(col+"_Y")

    df.drop(columns=["Unnamed: 0"], inplace=True)
    df.to_csv(file)

/tmp/ipykernel_6374/2636608381.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CandidatePosition"][index] = CIpca.transform([ci])[0]
/tmp/ipykernel_6374/2636608381.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NeighbourText"][index] = NTIpca.transform([nti])[0]
/tmp/ipykernel_6374/2636608381.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NeighbourX"][index] = NXIpca.transform([nxi])[0]
/tmp/ipykernel_6374/

In [12]:
df = pd.read_csv("10.jpeg.csv")
df

,Unnamed: 0,text,x1,y1,x2,y2,Class,invoice,invoice_X,invoice_Y,...,mre,mre_X,mre_Y,offi,offi_X,offi_Y,CandidatePosition,NeighbourText,NeighbourX,NeighbourY
0,0,31-Dec-22,0.750742,0.184506,0.826901,0.193357,invoice_date,-0.038794,0.157158,0.001049,...,0,0,0,0,0,0,-0.522747,0.002966,-0.079655,-0.014344
1,1,+91,0.329160,0.627065,0.359365,0.636737,invoice_date,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0.339365,-0.015144,-0.120018,-0.011804


In [13]:
def processName(text):
    if text[1]=="_":
        return (text[0]+".jpeg")
    elif text[2]=="_":
        return (text[:2]+".jpeg")
    else:
        return (text[:3]+".jpeg")

In [14]:
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Dataset")
dataset = pd.read_csv("_annotations.csv")
dataset = dataset[dataset["class"]=="InvoiceDate"]
dataset["Encoding"] = 0
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
for index in dataset.index:
    dataset["filename"][index] = processName(dataset["filename"][index])
    img = cv2.imread(dataset["filename"][index])
    dataset["xmin"][index]/=img.shape[1]
    dataset["xmax"][index]/=img.shape[1]
    dataset["ymin"][index]/=img.shape[0]
    dataset["ymax"][index]/=img.shape[0]
    arr = []
    arr.append(dataset["xmin"][index])
    arr.append(dataset["ymin"][index])
    arr.append(dataset["xmax"][index])
    arr.append(dataset["ymax"][index])
    encoding = CIpca.transform([arr])
    dataset["Encoding"][index] = encoding
dataset

/tmp/ipykernel_6374/3341279578.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["filename"][index] = processName(dataset["filename"][index])
/tmp/ipykernel_6374/3341279578.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["xmin"][index]/=img.shape[1]
/tmp/ipykernel_6374/3341279578.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["xmax"][index]/=img.shape[1]
/tmp/ipykernel_6374/3341279578.py:11: SettingW

,filename,width,height,class,xmin,ymin,xmax,ymax,Encoding
0,68.jpeg,2550,3300,InvoiceDate,0.682353,0.163636,0.774510,0.189394,-0.458796
4,56.jpeg,2550,3300,InvoiceDate,0.047451,0.142424,0.119608,0.157273,0.303090
7,100.jpeg,1134,1566,InvoiceDate,0.836861,0.236909,0.940917,0.256705,-0.601965
9,20.jpeg,2525,3500,InvoiceDate,0.621782,0.068571,0.697030,0.085714,-0.447553
12,76.jpeg,2550,3300,InvoiceDate,0.221176,0.140303,0.297255,0.161212,0.090908
...,...,...,...,...,...,...,...,...,...
306,5.jpeg,2481,3508,InvoiceDate,0.607013,0.167332,0.723096,0.185576,-0.382268
310,47.jpeg,3508,2481,InvoiceDate,0.314709,0.252318,0.373147,0.272471,0.069904
313,19.jpeg,2550,3300,InvoiceDate,0.605490,0.170606,0.683922,0.184848,-0.356376
315,7.jpeg,2479,3500,InvoiceDate,0.819282,0.158286,0.865672,0.171714,-0.604957


In [16]:
def annotate(file):
    df = pd.read_csv(file)
    df.drop_duplicates(keep="first", inplace=True)
    fileName = file.split(".")
    imageName = fileName[0] + "." + fileName[1]
    data = dataset[dataset["filename"]==imageName]
    df["Similarity"] = 100
    if len(data["Encoding"].to_list())>0:
        true = data["Encoding"].to_list()[0]
        for index in df.index:
            df["Similarity"][index] = abs(true - df["CandidatePosition"][index])
        df["Output"] = 0
        for index in df.index:
            if df["Similarity"][index] == df["Similarity"].min():
                df["Output"][index] = 1
        return df
    else:
        true = 0
        for index in df.index:
            df["Similarity"][index] = abs(true - df["CandidatePosition"][index])
        df["Output"] = 0
        return df


In [17]:
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset")

In [18]:
for file in datasetDir:
    output = annotate(file)
    output.to_csv(file)

/tmp/ipykernel_6374/3504863772.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Similarity"][index] = abs(true - df["CandidatePosition"][index])
/tmp/ipykernel_6374/3504863772.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Output"][index] = 1
/tmp/ipykernel_6374/3504863772.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Output"][index] = 1
/tmp/ipykernel_6374/3504863772.py:11: SettingWithCopyWarning: 
A value

In [21]:
sample = pd.read_csv("2.jpeg.csv")
sample.drop(columns=["Unnamed: 0.1", "Unnamed: 0", "Similarity"], inplace=True)
sample

,text,x1,y1,x2,y2,Class,invoice,invoice_X,invoice_Y,pitstop,...,mre_X,mre_Y,offi,offi_X,offi_Y,CandidatePosition,NeighbourText,NeighbourX,NeighbourY,Output
0,22-Sep-22,0.151674,0.138699,0.217023,0.148402,invoice_date,0.000000,0.000000,0.000000,0,...,0,0,0,0,0,0.176430,-0.015177,-0.121312,-0.012145,0
1,19-Sep-22,0.726906,0.282534,0.799516,0.292523,invoice_date,-0.038794,0.204518,0.012842,0,...,0,0,0,0,0,-0.419750,0.017715,0.044817,0.003095,1
2,1-Oct-22,0.664784,0.554509,0.718031,0.561358,invoice_date,0.000000,0.000000,0.000000,0,...,0,0,0,0,0,-0.135298,-0.004150,-0.109010,-0.009620,0
